In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import numpy as np
import zipfile
import random
import shutil
import os

In [20]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
# from tensorflow.keras.optimizers import Adam

# # Load the pre-trained ResNet50 model without the top layer (so you can add your own)
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# # Freeze the layers in the base model (to retain pre-trained weights)
# for layer in base_model.layers:
#     layer.trainable = False

# # Add custom top layers for your task
# x = base_model.output
# x = GlobalAveragePooling2D()(x)  # Global average pooling
# x = Dense(1024, activation='relu')(x)  # Fully connected layer
# x = Dense(9, activation='softmax')(x)  # Assuming 9 classes in your dataset

# # Define the final model
# model = Model(inputs=base_model.input, outputs=x)

# # Compile the model
# model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # model.summary()

In [11]:
# model.summary()

In [21]:
resnet_model = Sequential()

pretrained_model = ResNet50(include_top=False,
                            input_shape=(150,150,3),
                            pooling='avg', classes=9,
                            weights='imagenet')

for layer in pretrained_model.layers:
    layer.trainable = False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(9, activation='softmax'))

In [22]:
resnet_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 2048)           │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 9)              │         4,617 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,641,417 (94.00 MB)

 Trainable params: 1,053,705 (4.02 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [23]:
# Compile the model
learning_rate = 0.0001
optimizer = Adam(learning_rate=learning_rate)

resnet_model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
base_dir = 'splitted_balanced_dataset'

In [26]:
train = ImageDataGenerator(rescale=1/255)
validation = ImageDataGenerator(rescale=1/255)

In [27]:
train_dataset = train.flow_from_directory(base_dir+"/train",
                                          target_size=(150, 150),
                                          batch_size=32,
                                          class_mode='sparse')

validation_dataset = validation.flow_from_directory(base_dir+"/val",
                                          target_size=(150, 150),
                                          batch_size=32,
                                          class_mode='sparse')

Found 2484 images belonging to 9 classes.
Found 828 images belonging to 9 classes.


In [28]:
# Train the model
history = resnet_model.fit(train_dataset, epochs=20, validation_data=validation_dataset)

c:\Projects\python\Pattern Assignment 3\.venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.1250 - loss: 2.2756 - val_accuracy: 0.1316 - val_loss: 2.1745
Epoch 2/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.1804 - loss: 2.1676 - val_accuracy: 0.1836 - val_loss: 2.1412
Epoch 3/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 78s 994ms/step - accuracy: 0.1827 - loss: 2.1412 - val_accuracy: 0.1787 - val_loss: 2.1279
Epoch 4/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.2117 - loss: 2.1233 - val_accuracy: 0.2548 - val_loss: 2.1034
Epoch 5/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 78s 997ms/step - accuracy: 0.2508 - loss: 2.0956 - val_accuracy: 0.2766 - val_loss: 2.0763
Epoch 6/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 78s 1000ms/step - accuracy: 0.2656 - loss: 2.0774 - val_accuracy: 0.2995 - val_loss: 2.0537
Epoch 7/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.2709 - loss: 2.0587 - val_accuracy: 0.3068 - val_loss: 2.0310
Epoch 8/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 77s 993ms/step - accuracy: 0.2808 - loss: 2.0370 - val_accuracy: 0.2